In [2]:
import pandas as pd

In [13]:
df = pd.read_json('fruits.json')
df = df['Fruits'].apply(pd.Series) #get the dictionaries in Fruits as columns

df

,fruitName,fruitColor,price ($)
0,Apple,Red,1.50
1,Orange,Orange,2.00
2,Banana,Yellow,0.50
3,Grape,Purple,0.10
4,Mango,RedOrange,1.50
5,Lemon,Yellow,1.00
6,Lime,Green,1.00
7,Peach,YellowOrange,1.75
8,Watermelon,"Green, Red",1.00
9,Cherry,Red,0.10
